In [1]:
import pygmtools as pygm
import torch
import numpy as np

pygm.set_backend('pytorch')

In [5]:
arr = torch.Tensor(
    [[1, 1, 2], [2, 3, 4], [1, 1, 1]]
)
arr

tensor([[1., 1., 2.],
        [2., 3., 4.],
        [1., 1., 1.]])

In [6]:
y = pygm.sinkhorn(arr)
y

tensor([[0.3608, 0.2785, 0.3608],
        [0.1719, 0.3608, 0.4673],
        [0.4673, 0.3608, 0.1719]])

In [8]:
print('sum:', y.sum())
print('row sum', y.sum(dim=1))
print('col sum', y.sum(dim=0))

sum: tensor(3.)
row sum tensor([1.0000, 1.0000, 1.0000])
col sum tensor([1., 1., 1.])


In [9]:
(y * arr).sum()

tensor(5.6561)